<a href="https://colab.research.google.com/github/akashmanna/try-pytorch-lightning/blob/main/Simple_FMNIST_Pytorch_Lightening.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [40]:
!pip install pytorch-lightning

In [41]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
import pytorch_lightning as pl

In [42]:
from torch.nn import Conv2d, MaxPool2d, Linear 

class simpleCNN(pl.LightningModule):
    def __init__(self):
        super().__init__()

        self.model= nn.Sequential(
            Conv2d(in_channels=1, out_channels=6, kernel_size=5),
            MaxPool2d(kernel_size=2, stride=2),
            nn.ReLU(),
            Conv2d(in_channels=6, out_channels=12, kernel_size=5),
            MaxPool2d(kernel_size=2, stride=2),
            nn.ReLU(),
            nn.Flatten(start_dim=1),
            nn.Linear(in_features=12*4*4, out_features=120),
            nn.ReLU(),
            nn.Linear(in_features=120, out_features=60),
            nn.ReLU(),
            nn.Linear(in_features=60, out_features=10)
        )

    def forward(self, x):
        return F.softmax(self.model(x))

    def training_step(self, batch, batch_idx):
        x, y = batch
        # print(x.shape)
        # x = x.view(x.size(0), -1)
        x = self.model(x)

        # y = y.cuda()
        # y = F.one_hot(y, num_classes=10).cuda()
        # y=y.type(torch.BoolTensor).cuda()

        ce_loss = torch.nn.CrossEntropyLoss()
        loss = ce_loss(x,y)
        # loss = torch.nn.BCEWithLogitsLoss(x,y)
        self.log('train_loss', loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-2)
        return optimizer



In [43]:
!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz

transform_train = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize((0.5), (0.5))
                    ])

train_data = MNIST(root = './', train=True, download=True, transform=transform_train)
val_data = MNIST(root = './', train=False, download=True, transform=transform_train)

--2021-03-29 14:07:48--  http://www.di.ens.fr/~lelarge/MNIST.tar.gz
Resolving www.di.ens.fr (www.di.ens.fr)... 129.199.99.14
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.di.ens.fr/~lelarge/MNIST.tar.gz [following]
--2021-03-29 14:07:48--  https://www.di.ens.fr/~lelarge/MNIST.tar.gz
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘MNIST.tar.gz.2’

MNIST.tar.gz.2          [            <=>     ]  33.20M  10.5MB/s    in 3.2s    

2021-03-29 14:07:52 (10.5 MB/s) - ‘MNIST.tar.gz.2’ saved [34813078]

MNIST/
MNIST/raw/
MNIST/raw/train-labels-idx1-ubyte
MNIST/raw/t10k-labels-idx1-ubyte.gz
MNIST/raw/t10k-labels-idx1-ubyte
MNIST/raw/t10k-images-idx3-ubyte.gz
MNIST/raw/train-images-idx3-ubyte
MNIST/raw/train-labels-idx1-ubyte.gz
MNIST/raw/t10k-images-idx3-ubyte
MNIST/raw

In [44]:
train, val = random_split(train_data, [55000, 5000])


In [46]:
model  = simpleCNN()
trainer = pl.Trainer(max_epochs=4)
trainer.fit(model, DataLoader(train, batch_size=128), DataLoader(val, batch_size=128))

GPU available: False, used: False
TPU available: None, using: 0 TPU cores
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: you passed in a val_dataloader but have no validation_step. Skipping validation loop
  warnings.warn(*args, **kwargs)

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 33.0 K
-------------------------------------
33.0 K    Trainable params
0         Non-trainable params
33.0 K    Total params
0.132     Total estimated model params size (MB)


1